# 2. Projects operations system


Este proyecto realiza operaciones sobre los mails y los etiqueta segun el proyecto y el objeto del mail. 
En este caso, lee un email, identifica de que proyecto se trata, identifica cual es el objeto del email y realiza la acción indicada en cada caso

1. Utiliza Langchain agents.
1. Utiliza Tools
1. Utiliza langflow para el logging
1. No requiere subscriciones, Utiliza solamente modelos de LLM locales y ligeros, en este caso usamos qwen3:7b sobre ollama.


In [1]:
# %%capture --no-stderr
%pip install -U --upgrade pip langchain dotenv langchain_ollama langfuse pandas langgraph mcp langchain_mcp_adapters

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langfuse.langchain import CallbackHandler
from dotenv import load_dotenv
import os

# load environment variables from .env file
load_dotenv()
workfolder = os.getenv('WORKFOLDER')

# Initialize Langfuse CallbackHandler for LangGraph/Langchain (tracing)
langfuse_handler = CallbackHandler() 
llm_config = {"configurable": {"thread_id": "abc123"}, "recursion_limit": 20, "callbacks": [langfuse_handler]}

In [3]:
import pandas as pd

projects = [
    {
        "tag": "ATCLIBOT",
        "name": "Chatbot de Atención al Cliente Potenciado por IA",
        "description": "Chatbot de atención al cliente potenciado por IA que pueda manejar una amplia gama de consultas de clientes y proporcionar respuestas instantáneas. \
            El chatbot se integrará con el sistema de atención al cliente existente de la empresa y utilizará el procesamiento del lenguaje natural (PNL) para comprender y responder a las consultas de los clientes con precisión. \
            El chatbot será entrenado con datos históricos de atención al cliente para garantizar que pueda manejar los problemas comunes de manera efectiva.",
        "status": "Actualmente entrenando al chatbot con las últimas transcripciones de servicio al cliente.",
    },
    {
        "tag": "ECOMONITOR",
        "name": "Plataforma de Monitoreo de Energía Renovable",
        "description": "Plataforma de monitoreo de energía renovable que permite a los usuarios rastrear su consumo y producción de energía en tiempo real. \
            La plataforma incluirá características como análisis de uso de energía, seguimiento del rendimiento de los paneles solares y monitoreo del almacenamiento de baterías. \
            La plataforma estará diseñada para ser fácil de usar y proporcionará información práctica para ayudar a los usuarios a reducir su consumo de energía y disminuir su huella de carbono.",
        "status": "Realización de pruebas de usuario en la interfaz de la plataforma y la visualización de datos.",
    },
    {
        "tag": "DEVLEARN",
        "name": "Plataforma de Aprendizaje en Línea para Programación",
        "description": "Plataforma de aprendizaje en línea para programación que proporciona cursos interactivos, desafíos de codificación y retroalimentación en tiempo real a los estudiantes. \
            La plataforma estará diseñada para ser accesible a usuarios de todos los niveles de habilidad, desde principiantes hasta programadores avanzados. \
            Incluirá características como tutoriales en video, ejercicios de codificación y un foro comunitario para que los estudiantes se conecten y colaboren.",
        "status": "Implementando nuevos desafíos de codificación para mejorar la participación del usuario.",
    }
]

email_reasons = [
    {
        "reason": "UPDATE",
        "description": "A task of the project has finished",
        "action": "update the project status and notify the PM by email with the new status",
    }, 
    {
        "reason": "PROBLEM",
        "description": "Something has occurred that may delay the project",
        "action": "ask the analyst and technician to investigate the problem by email and put the PM in copy"
    },
    {
        "reason": "REQUEST",
        "description": "Something has been requested that changes the project",
        "action": "ask the analyst by email to evaluate the changes and ask the PM for approval"
    },
]

pd.DataFrame(projects).to_csv(os.path.join(workfolder, "projects.csv"), index=False)
pd.DataFrame(email_reasons).to_csv(os.path.join(workfolder, "email_reasons.csv"), index=False)


In [4]:
from mcp_pm_operations import get_emails, get_projects, get_reasons, modify_email, send_email

pm_operations_tools = [get_emails, get_projects, get_reasons, modify_email, send_email]
pm_operations_tools

[StructuredTool(name='get_emails', description="Returns the last email received by the system with empty tags, in json: {'id': 0, 'date': '', 'from_email': '', 'to_email': '', 'tags': '', 'subject': '', 'body': ''}.\n   If no emails with empty tags are found, returns a no new message error.", args_schema=<class 'langchain_core.utils.pydantic.get_emails'>, func=<function get_emails at 0x77c2a47f6200>),
 StructuredTool(name='get_projects', description="Returns the list of the projects, in json: [{'tag':'','name':'','description':''}]", args_schema=<class 'langchain_core.utils.pydantic.get_projects'>, func=<function get_projects at 0x77c2a47f6d40>),
 StructuredTool(name='get_reasons', description="Returns the list of reasons for sending an email, in json: [{ 'reason':'','description':'','action':'' }].", args_schema=<class 'langchain_core.utils.pydantic.get_reasons'>, func=<function get_reasons at 0x77c2a47f7920>),
 StructuredTool(name='modify_email', description='Modifies an email identi

In [5]:
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent

simple_agent = create_react_agent(name="simple_agent", model=ChatOllama(model="qwen3"), 
    tools=pm_operations_tools, response_format='json', prompt="You are a helpful assistant.")


In [6]:
import random 
project = random.sample(get_projects.invoke(''), 1)[0]
reason = random.sample(get_reasons.invoke(''), 1)[0]

input_message = {"role": "user", 
"content": f"""
send an email about a creative 100 word issue related to a project. 
- issue reason: {reason['description']}, 
- project description: {project['description']}, 

Do not explain the project description, just explain what part of the project is affected
Use profesional greeting, signature, farewell in the email, write in multiple lines.

email information:
- from_name: Jonh Doe
- from_email: jonh.doe@mycompany.com
- to: self@mycompany.com
- email language: Spanish
/no_think"""
        }

# Use the agent
for step in simple_agent.stream({"messages": [input_message]}, llm_config, stream_mode="values"):
    step["messages"][-1].pretty_print()


================================ Human Message =================================


send an email about a creative 100 word issue related to a project. 
- issue reason: Something has occurred that may delay the project, 
- project description: Plataforma de aprendizaje en línea para programación que proporciona cursos interactivos, desafíos de codificación y retroalimentación en tiempo real a los estudiantes.             La plataforma estará diseñada para ser accesible a usuarios de todos los niveles de habilidad, desde principiantes hasta programadores avanzados.             Incluirá características como tutoriales en video, ejercicios de codificación y un foro comunitario para que los estudiantes se conecten y colaboren., 

Do not explain the project description, just explain what part of the project is affected
Use profesional greeting, signature, farewell in the email, write in multiple lines.

email information:
- from_name: Jonh Doe
- from_email: jonh.doe@mycompany.com
- to: self@

In [7]:
input_message = {"role": "user", "content": """if there are no new emails you are done.
1. get the last email. 
2. get all projects. 
3. check each project against the email content. 
4. modify the email tag with the project tag. 
5. get the reasons list. 
6. decide wich reason is the email for. 
7. modify the email tag with the reason. 
""" }

# Use the agent
for step in simple_agent.stream({"messages": [input_message]}, llm_config, stream_mode="values"):
    step["messages"][-1].pretty_print()


================================ Human Message =================================

if there are no new emails you are done.
1. get the last email. 
2. get all projects. 
3. check each project against the email content. 
4. modify the email tag with the project tag. 
5. get the reasons list. 
6. decide wich reason is the email for. 
7. modify the email tag with the reason. 

================================== Ai Message ==================================
Name: simple_agent

<think>
Okay, let's break down the user's query step by step. The user provided a list of instructions that need to be followed in order. Let me go through each step and see which functions I can use here.

First, the user says, "if there are no new emails you are done." So the first thing is to check for new emails. Looking at the tools provided, there's a function called get_emails that returns the last email with empty tags. If there are no such emails, it returns a no new message error. So I should start by callin

In [8]:

# import pandas as pd

# stakeholders = [
#     {"name": "Diana Miller", "phone": "555-456-7890", "email": "diana.miller@cityhall.com", "company": "cityhall", "vip": False, "prefered_languaje": "Italian", "role": "technician"},
#     {"name": "Henry Green", "phone": "555-890-1234", "email": "henry.green@cityhall.com", "company": "cityhall", "vip": True, "prefered_languaje": "Italian", "role": "analist"},
#     {"name": "Leo Orange", "phone": "555-234-6789", "email": "leo.orange@cityhall.com", "company": "cityhall", "vip": False, "prefered_languaje": "Italian", "role": "pm"},
# ]

# roles = ["cto", "pm", "analist", "technician", "arquitect"]
# random_stakeholders = list(map(lambda role: {"name": random.choice([p for p in stakeholders  if p["role"] == role])["name"], "role": role}, roles))

# for project in projects:
#     project["stakeholders"] = random_stakeholders

# stakeholders_df = pd.DataFrame(stakeholders)
# stakeholders_df.to_csv(os.path.join(workfolder, "stakeholders.csv"), index=False)
# stakeholders_df = pd.read_csv(os.path.join(workfolder, "stakeholders.csv"))



In [9]:
# import json
# import random

# def get_random_stakeholder(project):
#     stakeholders = project['stakeholders']
#     stakeholders = stakeholders.replace("'", '"')
#     stakeholders = json.loads(stakeholders)
#     stakeholder = random.choice(stakeholders)
#     return stakeholders_df[stakeholders_df["name"]==stakeholder["name"]].reset_index(drop=True).iloc[0].to_dict()

# get_random_stakeholder(projects_df.iloc[0])